## Authentication

Authentication for this notebook will be via IAM Identity Center (Formerly SSO). Once you have logged into SSO for this notebook, all AWS actions will take place under your SSO credentials. You will need access across the organization to run the assessment.

In [ ]:
import boto3, os
import botocore.exceptions
from jupyterirtools import sso
import ransomware

org_root_account = "383086473915"
configuration_aggregator = 'aggregator-dr1zj3ositog'
active_regions = [
    "us-east-1",  "us-east-2",  "us-west-1",  "us-west-2"
]
role = "Jupyter-IR-AdministratorAccess"
destination_bucket = "gillemi-gillemi"
destination_prefix = "jupyter-reports/"
export_to_security_hub = False

sso.login(role, org_root_account)



## Best Practices

* AWS Config Aggregators to aggregate AWS Config Data Accross all regions and accounts.
* User AWS Systems Manager to open SSH Sessions instead of unbound SSH into a jump host.
* Make sure the AWS Systems Manager Agent is running and connecting on all instances

### Check Organizational Configure
* Organizational Trail in CloudTrail, Write logs to a central account
* Capture Data Events for Lambda and S3
* Check that AWS Config is enabled for all regions and accounts
* GuardDuty Enabled for all regions and accounts


In [ ]:


ransomware.check_org_trail(org_root_account, role, active_regions)
ransomware.cloudtrail_s3_events(org_root_account, active_regions, role)
ransomware.check_aws_config(active_regions, role)
ransomware.check_guardduty(active_regions, role)

## Validate Inspector

Check each region to see if inspector is active.



In [ ]:
ransomware.inspector_check_organization(active_regions, role)

# Instance Management
1. Based on AWS SSM agent running on the intances
1. Vulnerbility Management by Inspector
1. Validate patch status via patch manager.

## Patch Management

The next step is to view the current patch status of the EC2 instances

In [ ]:


ransomware.check_patch_manager(active_regions, role)


## Check for public facing instances
The following step will use AWS Config to list any EC2 instances that have a public IP address.


In [ ]:

ransomware.check_public_ips(configuration_aggregator)

## DNS Logging
Check to see if DNS logging is turned on for all VPCs

In [ ]:

ransomware.check_dnslogging(active_regions, role)

## Check for SSH unrestricted SSH Inbound
Strongly encouraged to block inbound SSH for the CIDR range 0.0.0.0. This will check the config rule for Organization-Wide SSH access.

If you currently allow SSH, consider using AWS Systems Manage Session Manager.

In [ ]:
ransomware.check_restricted_ssh_config_rule(configuration_aggregator, role, active_regions)

In [ ]:

ransomware.save_report(destination_bucket, destination_prefix)